In [1]:
import numpy as np
from case_studies import *
from scipy.optimize import *

In [2]:
def back_track(f,df,x,p,a=1,c1=0.01,rho=0.5):
    while f(np.array(x+a*p)) > f(np.array(x)) + c1*a* p.T@df(np.array(x)): #Check if a fulfills sufficient decrease conditions
        a = a*rho #If not, we decrease a by a factor rho
    return(a)

In [404]:
def constrained_newton(x0, A, f, df, hf,iterations=10):
    """
    Estimates the local minimum of f using the Newton algorithm with equality constarints
    """
    x_k = x0
    for i in range(iterations):

        # Check if hessian is positive definit
        if np.all(np.linalg.eigvals(hf(x_k)) > 0):
            B = hf(x_k)

        else:
            #compute approx hessian
            values,vectors = np.linalg.eig(hf(x_k))
            for i in range(len(values)):
                B =+ np.abs(values[i]) * np.outer(vectors[i],vectors[i])

        #create the linear system
        matrix_A = np.block([[B, A.T], [A, np.zeros([A.shape[0],A.shape[0]])]])
 
        matrix_B = np.block([-df(x_k),np.zeros(A.shape[0])])

        #solve the linear system and extract pk
        solved = np.linalg.solve(matrix_A, matrix_B)

        p_k = solved[:A.shape[1]]

        lam = solved[A.shape[1:]]

        #if np.linalg.norm(df(x_k)+A*lam) < 10**(-10): #stopping criteria 
            #break

        # Compute the step size
        alpha_k = back_track(x=x_k, p=p_k, f=f, df=df)

        # Update the current solution
        x_k = x_k + alpha_k*p_k

    return -x_k,i+1


In [403]:
m = 2
n = 3

A = np.random.rand(m,n)
x = np.random.rand(n)
b = np.random.rand(1)

x0 = x - np.linalg.pinv(A) @ (A@x + b)

x_newton,iterations_newton = constrained_newton(x0=np.array(x0),A=A,f=f3,df=df3,hf=Hf3)

In [335]:
def constrained_steepest_descent(f,df,x,A,max_iterations=10000,rho=0.5):
    """
    f is function that is being optimized
    df is the gradient of f
    x is staring point
    """
    beta = 1
    M = np.identity(A.shape[1]) - A.T @ np.linalg.inv(A@A.T) @ A
    for i in range(0,max_iterations):
        pk = -M @ df(x) #we set the ray direction

        ak = back_track(f=f,df=df,x=x,p=pk,a=beta) #We find the step size

        x = x + ak*pk #we update x with the stepsize and direction

        lam = np.linalg.norm(np.linalg.inv(A@A.T)@A@df(x)) #we calculate lambda with the eq we found in theory


        #print("stop",np.linalg.norm(df(x)+np.inner(A,lam)))
        if np.linalg.norm(df(x)+np.inner(A,lam)) < 10**(-5) :
            break
    
        beta = ak/rho #We calculate the new initial step size

    return(-x,i+1)

In [330]:
m = 1
n = 3

A = np.random.rand(m,n)
x = np.random.rand(n)
b = np.random.rand(1)

x0 = x - np.linalg.pinv(A) @ (A@x + b)

x,iterations = constrained_steepest_descent(f=f3,df=df3,x=np.array(x0),A=A)

In [405]:
m = 19
n = 20

A = np.random.rand(m,n)
x = np.random.rand(n)
b = np.random.rand(1)

x0 = x - np.linalg.pinv(A) @ (A@x + b)

x_newton,iterations_newton = constrained_newton(x0=np.array(x0),A=A,f=f3,df=df3,hf=Hf3)
x_sd,iterations_sd = constrained_steepest_descent(f=f3,df=df3,x=np.array(x0),A=A)

print("Newton's algorithm:\n iterations:",iterations_newton,"point found:",x_newton)

print("Steepest descent algorithm:\n iterations:",iterations_sd,"point found:",x_sd)

Newton's algorithm:
 iterations: 20 point found: [-1.76376637e-01 -5.10623702e-01  6.81114083e-01 -9.85285213e-02
 -1.31120493e-01  4.42928369e-01 -1.27398499e-01 -3.47497363e-01
  3.42520283e-01 -7.15839225e-01 -2.64267277e-02  4.89363994e-01
  2.22871849e-01  3.26779612e-01  3.49151614e-01 -2.43140084e-05
 -2.22133194e-01  4.17676494e-01  4.04078016e-02  1.91865316e-01]
Steepest descent algorithm:
 iterations: 10000 point found: [-1.76388265e-01 -5.10511824e-01  6.81115220e-01 -9.85426320e-02
 -1.31181099e-01  4.42939955e-01 -1.27362399e-01 -3.47510600e-01
  3.42546117e-01 -7.15800678e-01 -2.64587822e-02  4.89371217e-01
  2.22793717e-01  3.26802212e-01  3.49112274e-01 -3.32421407e-05
 -2.22140613e-01  4.17608742e-01  4.04675718e-02  1.91941030e-01]


In [406]:
linear_constraints = {"type": "eq", "fun": lambda x: A.dot(x) - b}

opt = minimize(f3,x0=x0,constraints=linear_constraints)
print(opt.x)

[-1.76391052e-01 -5.10485008e-01  6.81115492e-01 -9.85460140e-02
 -1.31195626e-01  4.42942732e-01 -1.27353747e-01 -3.47513772e-01
  3.42552309e-01 -7.15791438e-01 -2.64664650e-02  4.89372948e-01
  2.22774989e-01  3.26807629e-01  3.49102844e-01 -3.53821462e-05
 -2.22142391e-01  4.17592502e-01  4.04818979e-02  1.91959177e-01]
